In [6]:
import os
import sys

import matplotlib.pyplot as plt  # type: ignore
import pandas as pd  # type: ignore
import seaborn as sns  # type: ignore

sys.path.append(os.path.abspath(".."))

# Fix the import path
from model.model import run

# Here are the available parameters you can set:
# 
# Core simulation parameters:
# - steps: Number of simulation steps to run (default: 250)
# - dt: Time step size (default: 0.02)
# - s_start: Initial sheep population (default: 100)
# - w_start: Initial wolf population (default: 10)
# - sheep_max: Maximum sheep capacity (default: 110)
# 
# Wolf behavior parameters:
# - no_ai: If True, wolves use fixed theta; if False, wolves use AI (default: False)
# - theta_star: Default theta value when no_ai=True (default: 0.5)
# - churn_rate: Percentage of wolves that update their theta each step (default: 0.05)
# - threads: Number of parallel LLM calls when using async mode (default: 5)
#
# Wolf population dynamics parameters:
# - beta: Predation rate (default: 0.1)
# - gamma: Wolf death rate (default: 1.5)
# - delta: Conversion efficiency (default: 0.75)
#
# Sheep population dynamics parameters:
# - alpha: Sheep growth rate (default: 1)
# - eps: Small value to prevent extinction (default: 0.0001)
#
# Output parameters:
# - save_results: Whether to save simulation results (default: True)
# - path: Directory to save results (default: "../data/results")

# Don't set no_ai here, it will be set in the models


# Donʻt set no_ai here, it will be set in the models
my_args = {
    "steps": 200,
    "theta_star": 0.5,
    "churn_rate": .2,
}


In [ ]:
# Run model with AI-enabled wolves
ai_results = run(
    **my_args,
    no_ai=False,
    save_results=True
)

# Print some information about the results
print(f"Final sheep population: {ai_results['sheep_history'][-1]}")
print(f"Final wolf population: {ai_results['wolf_history'][-1]}")
print(f"Thetas: {ai_results['average_theta_history']}")

In [ ]:
# plots

# Create a proper time steps array
time_steps = list(range(len(ai_results["sheep_history"])))

ai_df = pd.DataFrame(
    {
        "t": time_steps,
        "Sheep": ai_results["sheep_history"],
        "Wolves": ai_results["wolf_history"],
        "Avg Theta": ai_results["average_theta_history"],
    }
)

# Create a plot with dual y-axes
fig, ax1 = plt.subplots(figsize=(12, 7))

# Plot populations on left y-axis
sns.lineplot(
    data=pd.melt(ai_df, id_vars=["t"], value_vars=["Sheep", "Wolves"]),
    x="t",
    y="value",
    hue="variable",
    palette=["cadetblue", "darkred"],
    ax=ax1,
)

# Plot average theta on right y-axis
ax2 = ax1.twinx()
sns.lineplot(
    data=ai_df,
    x="t",
    y="Avg Theta",
    color="darkgreen",
    ax=ax2,
    linewidth=2,
)

# Set the y-axis limits for the right axis
max_theta = max(ai_results["average_theta_history"])
min_theta = min(ai_results["average_theta_history"])
theta_range = max_theta - min_theta
ax2.set_ylim(max(0, min_theta - 0.1 * theta_range), max_theta + 0.1 * theta_range)

# Add labels and legend
ax1.set_xlabel("Time Steps")
ax1.set_ylabel("Population")
ax2.set_ylabel("Average Theta")
ax1.legend(title="", loc="upper left", frameon=False)
ax2.legend(["Avg Theta"], loc="upper right", frameon=False)

plt.title("Wolf-Sheep Population Dynamics with AI-Enabled Wolves")
plt.tight_layout()

In [ ]:
# Run model with no_ai=True (note the correct keyword argument syntax)
results = run(
    **my_args,
    no_ai=True,
    save_results=True
    )

In [ ]:


# Create a proper time steps array
time_steps = list(range(len(results["sheep_history"])))

# Convert from results to a DataFrame for plotting
xdf = pd.DataFrame(
    {
        "t": time_steps,
        "Sheep": results["sheep_history"],                    # instead of .attribute
        "Wolves": results["wolf_history"],
        "Avg Theta": results["average_theta_history"],
    }
)

# Plot populations on left y-axis
ax1 = plt.gca()
sns.lineplot(
    data=pd.melt(xdf, id_vars=["t"], value_vars=["Sheep", "Wolves"]),
    x="t",
    y="value",
    hue="variable",
    palette=["cadetblue", "darkred"],
    ax=ax1,
)

# Plot average theta on right y-axis
ax2 = ax1.twinx()
sns.lineplot(
    data=xdf,
    x="t",
    y="Avg Theta",
    color="darkgreen",
    ax=ax2,
)

# Set the y-axis limits for the right axis to make theta more visible
# Scale to approximately 2x the max theta value
max_theta = max(results["average_theta_history"])
ax2.set_ylim(0, max_theta * 2)

ax2.set_ylabel("Average Theta")

# Customize the plot
ax1.set_xlabel("Time Steps")
ax1.set_ylabel("Population")
ax1.legend(title="", loc="upper left", frameon=False)
ax2.legend(["Avg Theta"], loc="upper right", frameon=False)

sns.despine(left=True, bottom=True)
plt.title("Wolf-Sheep Population Dynamics with Theta (No AI)")
plt.tight_layout()